In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder


In [7]:
dataset_path = "Data\\SOCOFing\\Real"
img_height, img_width = 96,96

def preprocess_images(img_path):
    img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_width,img_height))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    return img

def load_and_extract_features(dataset_path, model):
    images = []
    labels = []
    features = []
    for file in os.listdir(dataset_path):
        if file.endswith(".BMP"):
            img_path = os.path.join(dataset_path,file)
            img = preprocess_images(img_path)
            img = np.expand_dims(img, axis=0)
            img = np.expand_dims(img, axis=-1)
            feature = model.predict(img)
            features.append(feature.flatten())
            images.append(img)
            label = int(file.split("__")[0])-1
            labels.append(label)
    return np.array(features), np.array(labels)

In [3]:
def create_model():
    model = Sequential([
        Conv2D(32,(3,3), activation='relu', input_shape = (img_height,img_width,1)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        GlobalAveragePooling2D()
    ])
    return model
model = create_model()

d:\Project\Th-c-t-p-t-t-nghi-p\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Tạo mô hình MobileNetV2 để trích xuất đặc trưng
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights=None)
# Add global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Create the new model
model2 = Model(inputs=base_model.input, outputs=x)
features, labels = load_and_extract_features(dataset_path,model2)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
np.save('fingerprint_features.npy', features)
np.save('fingerprint_labels.npy', encoded_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━

In [5]:
def add_new_fingerprint(new_fingerprint_image_path, new_label):
    # Tiền xử lý ảnh vân tay mới
    new_img = preprocess_images(new_fingerprint_image_path)
    new_img = np.expand_dims(new_img, axis=0)
    new_img = np.expand_dims(new_img, axis=-1)

    # Trích xuất đặc trưng
    new_feature = model.predict(new_img).flatten()

    # Tải đặc trưng và nhãn hiện có
    features = np.load('fingerprint_features.npy')
    labels = np.load('fingerprint_labels.npy')

    # Mã hóa nhãn mới
    new_encoded_label = label_encoder.transform([new_label])[0]

    # Thêm đặc trưng và nhãn mới
    features = np.vstack([features, new_feature])
    labels = np.append(labels, new_encoded_label)

    # Lưu lại đặc trưng và nhãn
    np.save('fingerprint_features.npy', features)
    np.save('fingerprint_labels.npy', labels)


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def verify_fingerprint(fingerprint_image_path):
    # Preprocess the input fingerprint image
    input_image = preprocess_images(fingerprint_image_path)
    input_image = np.expand_dims(input_image, axis=0)

    # Extract features
    input_feature = model2.predict(input_image).flatten()

    # Load existing features and labels
    features = np.load('fingerprint_features.npy')
    labels = np.load('fingerprint_labels.npy')

    # Debug print: check the shapes and values of the features
    print("Input feature shape:", input_feature.shape)
    print("Database features shape:", features.shape)
    print("Input feature:", input_feature[:10])
    print("Database first feature:", features[0][:10])

    # Calculate cosine similarity between the input feature and database features
    similarities = cosine_similarity([input_feature], features)

    # Find the feature with the highest similarity
    best_match_index = np.argmax(similarities)
    best_match_label = labels[best_match_index]

    # Decode the label
    decoded_label = label_encoder.inverse_transform([best_match_label])[0]
    print("Best match label:", best_match_label)
    print("Best match similarity:", similarities[0][best_match_index])

    # Check the threshold to determine if it matches
    if similarities[0][best_match_index] > 0.9:
        print(f"Unlock successful! Matched label: {decoded_label}")
    else:
        print("Unlock failed! No matching fingerprint found.")

# Dùng hàm xác thực với ảnh vân tay đầu vào
verify_fingerprint("Data\\SOCOFing\\Altered\\Altered-Easy\\102__M_Left_index_finger_Obl.BMP")


UnboundLocalError: local variable 'image' referenced before assignment